In [1]:
import torch
import torchvision.transforms.functional as F
from src import model, dataset

In [3]:
dataset.CracksDataset.TRANSFORM['train']

Compose(
    SingleChannel()
    RandomVerticalFlip(p=0.5)
    RandomHorizontalFlip(p=0.5)
    RandomRotation()
    FiveCrop(size=(224, 224))
    ToTensor()
)

In [4]:
dataset.CracksDataset.TRANSFORM['valid']

Compose(
    SingleChannel()
    FiveCrop(size=(224, 224))
    ToTensor()
)

In [ ]:
dataset = dataset.CracksDataset('valid')
loader = dataset.get_loader(batch_size=2, num_workers=0)
batch = next(iter(loader))

In [ ]:
net = model.UNet()#.eval()
state_dict = torch.load('logs/segmentation/1/checkpoints/best.pth', map_location='cpu')
net.load_state_dict(state_dict['model_state_dict'])

In [ ]:
targets = batch.pop('masks')
inputs = batch.pop('images')
outputs = net(inputs)

In [ ]:
save_fmt = 'tests/{}.png'
for index, (inp, tar, out) in enumerate(zip(inputs, targets, outputs)):
    F.to_pil_image(inp).save(save_fmt.format(f'input_{index}'))
    F.to_pil_image(tar).save(save_fmt.format(f'target_{index}'))
    F.to_pil_image(out).save(save_fmt.format(f'predict_{index}'))
    # print(inp.shape, tar.shape, out.shape)

In [1]:
import torch
from src import model
net = model.VGGEncoderClassifier()
inputs = torch.rand(2, 3, 224, 224)
net(inputs)

torch.Size([2, 512, 14, 14])
torch.Size([2, 512, 14, 14])
torch.Size([2, 512, 7, 7])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x25088 and 50176x2048)